In [1]:
import os
import re
import sys
import glob

import numpy as np
import pandas as pd

import seaborn as sns

# add gaugi by hand
sys.path.append('../../.__python__/')
from Gaugi import load as gload
from kolmov import table_info

In [2]:
base_path = '/volume/'
task_name = 'v1_task/user.mverissi.task.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*'
task_full_name = os.path.join(base_path, task_name)

In [3]:
print(task_full_name)

/volume/v1_task/user.mverissi.task.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*


In [4]:
tuned_info = {
    # validation
    'max_sp_val'     : [],
    'max_sp_pd_val'  : [],
    'max_sp_fa_val'  : [],
    'auc_val'        : [],
    # operation
    'max_sp_op'     : [],
    'max_sp_pd_op'  : [],
    'max_sp_fa_op'  : [],
    'auc_op'        : [],
}

In [5]:
test = table_info(task_full_name, tuned_info, tag='v1')

There are 900 files for this task...


In [6]:
# fill table
test.fill_table()

Filling the table... 
End of fill step, a pandas DataFrame was created...


In [7]:
cv_df = test.get_pandas_table()
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
0,v1,2,0,8,0,40,tunedDiscr.jobID_0044.pic.gz,21991,226563,0.964129,0.971364,0.043079,0.989309,0.965064,0.974217,0.044045,0.990439
1,v1,2,0,8,0,41,tunedDiscr.jobID_0044.pic.gz,21991,226563,0.963296,0.972273,0.045639,0.988739,0.964953,0.972671,0.042734,0.990052
2,v1,2,0,8,0,42,tunedDiscr.jobID_0044.pic.gz,21991,226563,0.963471,0.970000,0.043035,0.989260,0.965217,0.973717,0.043246,0.990617
3,v1,2,0,8,0,43,tunedDiscr.jobID_0044.pic.gz,21991,226563,0.962072,0.977273,0.053010,0.989100,0.963790,0.972989,0.045365,0.990130
4,v1,2,0,8,0,44,tunedDiscr.jobID_0044.pic.gz,21991,226563,0.963316,0.970000,0.043344,0.989608,0.965221,0.975535,0.045038,0.990587


In [8]:
cv_df = test.filter_inits('max_sp_val')

In [9]:
len(cv_df)

1650

In [10]:
test.dump_table(cv_df, base_path, 'cv_table_v1')

In [16]:
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
69056,v1,0,0,0,0,46,tunedDiscr.jobID_0004.pic.gz,28455,205005,0.949406,0.963106,0.064195,0.982747,0.947130,0.959761,0.065418,0.982995
71325,v1,0,0,0,1,25,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.947411,0.962403,0.067463,0.981599,0.946596,0.957442,0.064189,0.982611
71344,v1,0,0,0,2,24,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.942160,0.945889,0.061561,0.980855,0.944615,0.958426,0.069096,0.982112
69569,v1,0,0,0,3,9,tunedDiscr.jobID_0000.pic.gz,28455,205005,0.946431,0.963106,0.070098,0.982423,0.946478,0.962643,0.069550,0.983515
68287,v1,0,0,0,4,37,tunedDiscr.jobID_0003.pic.gz,28455,205005,0.946725,0.962755,0.069171,0.981539,0.946067,0.961272,0.069018,0.983181
